In [14]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import datetime

import load_data

### One hidden layer

In [19]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([size_in, size_out])
        b = bias_variable([size_out])
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    

def train_model(data, label="tissue", learning_rate=1e-2):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}/".format(str(datetime.datetime.today().date()))
    N_IN = data.train.X.shape[1]
    N_OUT = data.train.y[label].shape[1]
    N_HIDDEN = int(np.mean(N_IN + N_OUT))
    
    x = tf.placeholder(tf.float32, [None, N_IN], name="x")
    y_true = tf.placeholder(tf.float32, [None, N_OUT], name="labels")
    hidden = tf.nn.relu(fc_layer(x, N_IN, N_HIDDEN), name="hidden")
    y_pred = fc_layer(hidden, N_HIDDEN, N_OUT, name="softmax")
    
    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=y_true, logits=y_pred), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + label)
    writer.add_graph(sess.graph)
    
    # training
    t0 = time.time()
    sess.run(tf.global_variables_initializer())

    for i in range(2001):
        batch_x, batch_y = data.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_x, y_true: batch_y[label]})
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ],
                feed_dict={x: tcga.train.X, y_true: tcga.train.y[label]})
    #         [test_accuracy, s] = sess.run([accuracy, summ],
    #             feed_dict={x: tcga.test.X, y_true: tcga.test.y["tissue"]})
            writer.add_summary(s, i)
            if i % 100 == 0:
                print("step", i, "training accuracy", train_accuracy)
    print("training time:", time.time() - t0)

In [20]:
tcga = load_data.read_data_sets("./data/mRNA_PCA_0.6_variance_MinMaxScaled.csv")
for label_name in ["tissue", "gender", "tumor"]:
    print(label_name)
    tcga.train.reset_epoch()
    train_model(tcga, label=label_name)

tissue
step 0 training accuracy 0.134239
step 100 training accuracy 0.727262
step 200 training accuracy 0.816486
step 300 training accuracy 0.867948
step 400 training accuracy 0.876583
step 500 training accuracy 0.889017
step 600 training accuracy 0.887175
step 700 training accuracy 0.886599
step 800 training accuracy 0.897191
step 900 training accuracy 0.899609
step 1000 training accuracy 0.903868
step 1100 training accuracy 0.885563
step 1200 training accuracy 0.90951
step 1300 training accuracy 0.906401
step 1400 training accuracy 0.908819
step 1500 training accuracy 0.910776
step 1600 training accuracy 0.901681
step 1700 training accuracy 0.910546
step 1800 training accuracy 0.902602
step 1900 training accuracy 0.906747
step 2000 training accuracy 0.902487
training time: 14.089812994003296
gender
step 0 training accuracy 0.488487
step 100 training accuracy 0.701934
step 200 training accuracy 0.710569
step 300 training accuracy 0.72116
step 400 training accuracy 0.720815
step 500 tr